The used dataset, is a public Kaggle dataset. [Dataset](https://www.kaggle.com/datasets/legendahmed/alzheimermridataset)

In [1]:
!unrar x -Y "/content/dataset.rar" "/content/" # Decompressing the dataset

Se han truncado las últimas 5000 líneas del flujo de salida.
Extracting  /content/dataset/train/NonDemented/nonDem1572.jpg             22%  OK 
Extracting  /content/dataset/train/NonDemented/nonDem1573.jpg             22%  OK 
Extracting  /content/dataset/train/NonDemented/nonDem1574.jpg             22%  OK 
Extracting  /content/dataset/train/NonDemented/nonDem1575.jpg             22%  OK 
Extracting  /content/dataset/train/NonDemented/nonDem1576.jpg             22%  OK 
Extracting  /content/dataset/train/NonDemented/nonDem1577.jpg             22%  OK 
Extracting  /content/dataset/train/NonDemented/nonDem1578.jpg             22%  OK 
Extracting  /content/dataset/train/NonDemented/nonDem1579.jpg             22%  OK 
Extracting  /content/dataset/train/NonDemented/nonDem158.jpg              22%  OK 
Extracting  /content/dataset/train/NonDemented/nonDem1580.jpg             22%  OK 
Extracting  /conten

In [9]:
# Important paths
train_path = '/content/dataset/train'
test_path = '/content/dataset/validation'

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.imagenet_utils import preprocess_input

width_shape = 48 # Image width
height_sape = 48 # Image height
num_labels = 4 # Number of classification labels
epochs = 15
batch_size = 100
label_names = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']

In [11]:
# Training and testing dataset configuration
training_dataset = ImageDataGenerator()
testing_dataset = ImageDataGenerator()

trainging_gen = training_dataset.flow_from_directory(
    train_path,
    target_size=(width_shape, height_sape),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical', shuffle=True)

testing_gen = testing_dataset.flow_from_directory(
    test_path,
    target_size=(width_shape, height_sape),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical', shuffle=True)

Found 5121 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.


In [12]:
# Defining the architecture
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization, Input, AveragePooling2D,Activation
from tensorflow.keras.optimizers import Adam

model = Sequential()

# Convolutional layer 1
model.add(Conv2D(32, kernel_size=(3, 3), padding="same", activation="relu", strides=1, input_shape = (width_shape, height_sape, 1)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# Convolutional layer 2
model.add(Conv2D(64, kernel_size=(5, 5), padding="same", activation="relu", strides=1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# Convolutional layer 3
model.add(Conv2D(128, kernel_size=(3, 3), padding="same", activation="relu", strides=1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# Convolutional layer 4
model.add(Conv2D(256, kernel_size=(3, 3), padding = "same", activation="relu", strides=1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# Data conversion
model.add(Flatten())

# Layer 6
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

# Classification
model.add(Dense(num_labels, activation='softmax'))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        320       
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 32)       128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 24, 24, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        51264     
                                                                 
 batch_normalization_1 (Batc  (None, 24, 24, 64)       2

In [14]:
# Compilation and optimization
optimizer = Adam(learning_rate=0.001, decay=1e-4 / epochs) # learning_rate=1e-4
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
# Decay parameter is how the learning_reate goes down in order not to miss the optimal point.
# In other words, the program has to decay learning rate to have more accurate steps by reducing the learning rate.

In [15]:
# TensorBoard
from tensorflow.keras.callbacks import TensorBoard
import datetime, os

%load_ext tensorboard
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
callbacks = TensorBoard(logdir, histogram_freq=1)

In [ ]:
# Training
model.fit(
    trainging_gen,
    epochs=epochs,
    validation_data=testing_gen,
    steps_per_epoch=trainging_gen.n//batch_size,
    validation_steps=testing_gen.n//batch_size,
    callbacks=[callbacks]
)

In [ ]:
score = model.evaluate(testing_gen, verbose=0)
# Returns the loss value and metrics values for the model in test mode
print("Score = ", score)

In [ ]:
# Performance graphic
%tensorboard --logdir logs

# Pink = validation
#Green = Train

In [ ]:
# Simple test
from tensorflow.keras.models import load_model
import cv2 
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt

test_faces = []

pic_path = "/content/dataset/validation/ModerateDemented/30.jpg"

test_face = cv2.cvtColor(cv2.imread(pic_path), cv2.COLOR_BGR2GRAY)
test_face = cv2.resize(test_face, (48, 48))
test_face2 = img_to_array(test_face)
test_face2 = np.expand_dims(test_face2, axis=0)
test_faces.append(test_face2)
prediction = model.predict(test_faces)

In [ ]:
print(label_names[np.argmax(prediction)])
plt.imshow(cv2.cvtColor(np.asarray(test_face), cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

In [ ]:
# Confusion matrix
from sklearn.metrics import confusion_matrix, f1_score, roc_curve, precision_score, recall_score, accuracy_score, roc_auc_score
from sklearn import metrics
from mlxtend.plotting import plot_confusion_matrix
from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt

test_data_gen = ImageDataGenerator()
testing_gen = test_data_gen.flow_from_directory(
    test_path,
    target_size=(width_shape, height_sape),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical', shuffle=False)

predictions = model.predict(testing_gen)
y_pred = np.random.binomial(1, .9, size=1000)
y_real = np.random.binomial(1, .9, size=1000)

confusion = confusion_matrix(y_real, y_pred)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix=confusion, display_labels=[False, True])
cm_display.plot()
plt.show()

# Quadrants of the confusion matrix:

Top-left -> True negative

Top-right -> False positive

Bottom-left -> False negative

Bottom-right -> True positive

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_real, y_pred))

In [ ]:
# Saving the model inside a "h5" file
model.save("EmotionDetector.h5")
# An "h5" file, is a file that contents a big amount of numeric, graphic and text data.

In [ ]:
# Accuracy calculation according to the confusion matrix results.
def accuracy_calculation(true_negative, false_positive, false_negative, true_positive):
  accuracy = (true_negative + true_positive) / (true_negative + false_positive + false_negative + true_positive)
  return accuracy * 100

def error_rate_calculation(true_negative, false_positive, false_negative, true_positive):
  error_rate = (false_positive + false_negative) / (true_negative + false_positive + false_negative + true_positive)
  return error_rate * 100

success_rate = accuracy_calculation(11, 86, 83, 820)
error_rate = error_rate_calculation(11, 86, 83, 820)

print("Accuracy = ", "{:.2f}".format(success_rate), "%")
print("Error rate = ", "{:.2f}".format(error_rate), "%")
